# Ryanair Datathon

Team: The Malviners

- This notebooks contains the final Data Cleaning, Feature Engineering, and Model after many versions created in the past days.
- Everything will be done using the train_extended dataset as we can further explore the data using the *variables known after prediction* and *estimations from Ryanair Models*

## Data Exploration + Outliers Cleaning

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import plotly.express as px
df = pd.read_csv("train_extended.csv")

### Calculate Distance Between Airports
- Distance between airports will be crucial for the model
- We use **airportsdata** library to get coordinates from all airports, and **geopy** to calculate the distance in miles between two airports
- SXF and SZY airports didn't have data on the library, so it is introduced manually

In [2]:
import airportsdata
airportsData = airportsdata.load('IATA')
from geopy import distance

df[['DepartureAirport','ArrivalAirport']] = df['ScheduledRoute'].str.split('-', expand=True)
def calculateGeoDistance(departure,arrival):
  try:
    if departure == 'SXF': 
      departure = (52.3733,13.5064)
      arrival = (airportsData[arrival]['lat'], airportsData[arrival]['lon'])
    elif departure == 'SZY': 
      departure = (53.4879,20.9465)
      arrival = (airportsData[arrival]['lat'], airportsData[arrival]['lon'])
    elif arrival == 'SXF': 
      arrival = (52.3733,13.5064)
      departure = (airportsData[departure]['lat'], airportsData[departure]['lon'])
    elif arrival == 'SZY': 
      arrival = (53.4879,20.9465)
      departure = (airportsData[departure]['lat'], airportsData[departure]['lon'])
    else:
      departure = (airportsData[departure]['lat'], airportsData[departure]['lon'])
      arrival = (airportsData[arrival]['lat'], airportsData[arrival]['lon'])
    dist = distance.distance(departure, arrival).miles
    return dist
  except: return 0

df['distance'] = df.apply(lambda x: calculateGeoDistance(x.DepartureAirport, x.ArrivalAirport), axis = 1)
df.head()

,FlightID,FlightNumber,AircraftRegistration,AircraftCapacity,AircraftTypeGroup,ServiceDescription,Carrier,AOCDescription,ScheduledRoute,ActualRoute,...,Infants,Bags,FlightTime,PlannedZeroFuelWeight,PlannedTOW,PlannedTripTime,TeledyneRampWeight,DepartureAirport,ArrivalAirport,distance
0,181142302,221,SPRZK,197.0,Max,Scheduled Flight,FR,Ryanair Sun,EMA-BUD,EMA-BUD,...,2.0,43.0,137.0,56242.0,62723.0,8246.0,62075.0,EMA,BUD,983.168625
1,178908306,3373,9HQEC,189.0,NG,Scheduled Flight,FR,Malta Air,TRN-MLA,TRN-MLA,...,0.0,7.0,102.0,57148.0,63878.0,6227.0,62002.0,TRN,MLA,737.477336
2,178886981,2814,9HQCN,189.0,NG,Scheduled Flight,FR,Malta Air,STN-CGN,STN-CGN,...,0.0,19.0,50.0,53620.0,58089.0,3653.0,58029.0,STN,CGN,306.931098
3,180538798,6893,SPRKP,189.0,NG,Scheduled Flight,FR,Ryanair Sun,DTM-KTW,DTM-KTW,...,1.0,8.0,71.0,53396.0,58999.0,4784.0,56070.0,DTM,KTW,504.890714
4,178863684,2156,EIDWJ,189.0,NG,Scheduled Flight,FR,Ryanair DAC,BHX-AGP,BHX-AGP,...,0.0,47.0,155.0,59199.0,68573.0,9275.0,68315.0,BHX,AGP,1097.743522


### DryEmptyWeight for each AircraftTypeGroup

- Values of DryEmptyWeight were taken from Ryanair's powerpoint.
- For NG type there are planes with 2 capacities, so we introduce the weight by `AircraftCapacity`.

In [3]:
dWeights = {197:45070, 189: 41140, 148:37648 ,180:42600}
df['DryEmptyWeight'] = df['AircraftCapacity'].apply(lambda x: dWeights[x])

### Flights that had their ScheduledRoute changed
- There are 618 flights
- If the routes are changed in the middle of the flight, it should not affect the TeledyneRampWeight.
- These routes change significantly BlockTimeScheduled from the actual **BlockTime** (see boxplot), while **distance** flought will also be affected. These two variables will be very relevant to estimate the **TeledyneRampWeight**
- There is no way of predicting if the route will be changed. We choose to remove these rows from the data.

In [4]:
df['diffBlockTime'] = df['BlockTimeScheduled'] - df['BlockTime']
px.box(df.loc[df.ScheduledRoute != df.ActualRoute,'diffBlockTime'], title='Difference in BlockTime (Planned and Actual) when route is changed')

In [5]:
# Keep only non-changed routes.
df = df.loc[df.ScheduledRoute == df.ActualRoute,:]

### Flight Times
- Largest routes stay below 400 minutes `FlightTime`. 
- Above 400 min we find outliers. Wrong inputations probably. We drop these.

In [6]:
df['PlannedTripTime'] = df['PlannedTripTime']/60
df.loc[df.FlightTime > 400, ['ScheduledRoute','PlannedTripTime','FlightTime','distance','BlockTime']].sort_values('FlightTime')

,ScheduledRoute,PlannedTripTime,FlightTime,distance,BlockTime
83730,LPL-DUB,37.650000,1468.0,141.556097,43.0
345339,STN-FKB,63.916667,1500.0,407.780570,77.0
758571,BDS-MXP,109.600000,1527.0,578.180982,102.0
300085,BVA-ALC,116.600000,1550.0,782.648183,124.0
618059,KRK-STN,121.633333,1559.0,859.504205,140.0
726402,BRI-BVA,139.433333,1576.0,913.459829,154.0
698059,WMI-EDI,152.283333,1579.0,998.239064,163.0
204748,LIS-STN,153.550000,1584.0,1011.565340,162.0
501644,WRO-DUB,151.650000,1585.0,990.836416,157.0
61899,LUX-LIS,153.683333,1586.0,1064.601590,164.0


In [7]:
df = df.loc[df.FlightTime < 400,:]

- There are 121 flights where `FlightTime` >= `BlockTime` (time from departure to arrival including taxis).
- Either `FlightTime` or `BlockTime` are wrongly inputed.
- The FlightTime is also clearly higher than the route's median FlightTime as well as ryanair's model `PlannedTripTime` (accounting only for flight time).
- Drop these.

In [8]:
df.loc[df.FlightTime >= df.BlockTime, ['distance','PlannedTripTime','FlightTime','BlockTime','BlockTimeScheduled']]

,distance,PlannedTripTime,FlightTime,BlockTime,BlockTimeScheduled
7452,599.944664,95.200000,153.0,107.0,105.0
11697,1450.857639,197.650000,246.0,200.0,210.0
19456,813.308658,119.100000,171.0,141.0,135.0
20129,470.787594,80.050000,133.0,85.0,95.0
38746,922.133841,135.216667,188.0,144.0,150.0
...,...,...,...,...,...
755089,926.938504,148.250000,193.0,153.0,170.0
755205,718.538742,107.516667,158.0,114.0,120.0
758394,799.784025,113.866667,167.0,122.0,135.0
759797,363.498477,63.433333,117.0,69.0,75.0


In [9]:
df = df.loc[df.FlightTime <= df.BlockTime,:]

- There are 28 flights with `FlightTime` == 0
- If these are flights that couldn't start their flight for some reason, this will introduce noise for the prediction of the `TeledyneRampWeight`
- Drop these

In [10]:
df.loc[df.FlightTime == 0,:].shape
df = df.loc[df.FlightTime > 0,:]

- We calculate the `flightTimeRatio`to see outliers of `FlightTime`
- Those that have a FlightTime of more than 30% of the AvgFlightTime on that route are dropped (1445)
- Those that have a FlightTime less than 80% of the AvgFlightTime on that route are dropped (874)

In [11]:
# Calculate average flight time per route
df['avgFlighTimeRoute'] = df['ScheduledRoute'].map(df.groupby('ScheduledRoute')['FlightTime'].mean())
# Calculate number of flights per route
df['numberOfFlights'] = df['ScheduledRoute'].map(df.groupby('ScheduledRoute')['FlightID'].count())
# Calculate FlightTime/avgFlighTimeRoute
df['flightTimeRatio'] = df['FlightTime'] / df['avgFlighTimeRoute']
df.loc[df.flightTimeRatio < 0.8,['FlightTime','avgFlighTimeRoute']].sort_values(by='FlightTime')

,FlightTime,avgFlighTimeRoute
17880,1.0,73.134875
348045,1.0,22.057692
98538,1.0,53.655570
280048,2.0,63.471299
246890,3.0,62.040234
...,...,...
388217,197.0,246.756757
96956,201.0,251.741697
211125,208.0,261.793103
163066,209.0,262.321739


In [12]:
df = df.loc[df.flightTimeRatio > 0.8,:]
df = df.loc[df.flightTimeRatio < 1.3,:]

### Taxis
- TaxiOut might vary a lot. Depends on DepartureAirport
- All flights with 2.5*STD of their mean TaxiOut are dropped.

In [13]:
taxiOut = df.groupby('DepartureAirport')['TaxiOut'].agg(['count','mean','median','std']).reset_index()
px.scatter(taxiOut, x='DepartureAirport',y='median', title='Median TaxiOut per DepartureAirport')

In [14]:
taxiOut['thresholdUpTaxi'] = taxiOut['mean'] + 2.5*taxiOut['std']
taxiOut['thresholdDownTaxi'] = taxiOut['mean'] - 2.5*taxiOut['std']
df = df.merge(taxiOut.loc[:,['DepartureAirport','thresholdUpTaxi','thresholdDownTaxi']], on='DepartureAirport', how='left')
df.loc[(df.TaxiOut < df.thresholdDownTaxi) | (df.TaxiOut > df.thresholdUpTaxi),['TaxiOut','thresholdUpTaxi','thresholdDownTaxi']]

,TaxiOut,thresholdUpTaxi,thresholdDownTaxi
7,19.0,17.628796,2.569264
73,19.0,14.842126,4.219857
110,48.0,26.780572,-5.504608
146,25.0,22.074887,-3.575277
245,32.0,19.821282,3.097823
...,...,...,...
761719,15.0,14.470427,1.835159
761870,30.0,26.408146,-3.301113
761878,29.0,23.881982,0.628938
761958,26.0,21.053666,1.631471


In [15]:
# Drop TaxiOuts
df = df.loc[(df.TaxiOut > df.thresholdDownTaxi) | (df.TaxiOut < df.thresholdUpTaxi),:]

- TaxiIn -> calculated from (BlockTime - TaxiOut - FlightTime)
- Depends on Arrival Airport
- Same as for TaxiOut -> inputation of median for 2.5*std

In [16]:
df['TaxiIn'] = df['BlockTime'] - df['TaxiOut'] - df['FlightTime']
taxiIn = df.groupby('ArrivalAirport')['TaxiIn'].agg(['count','mean','median','std']).reset_index()
px.scatter(taxiIn, x='ArrivalAirport',y='median', title='Median TaxiIn per ArrivalAirport')

In [17]:
taxiIn['thresholdUpTaxiIn'] = taxiIn['mean'] + 2.5*taxiIn['std']
taxiIn['thresholdDownTaxiIn'] = taxiIn['mean'] - 2.5*taxiIn['std']
df = df.merge(taxiIn.loc[:,['ArrivalAirport','thresholdUpTaxiIn','thresholdDownTaxiIn']], on='ArrivalAirport', how='left')
df.loc[(df.TaxiIn < df.thresholdDownTaxiIn) | (df.TaxiIn > df.thresholdUpTaxiIn),['TaxiIn','thresholdUpTaxiIn','thresholdDownTaxiIn']]

,TaxiIn,thresholdUpTaxiIn,thresholdDownTaxiIn
217,20.0,11.690191,-1.295180
540,12.0,10.525677,-0.598480
551,12.0,9.392175,-0.240757
555,65.0,12.263020,-1.921264
596,15.0,8.782033,-0.013173
...,...,...,...
761498,63.0,8.584560,-0.662308
761590,7.0,6.546061,-0.071292
761708,14.0,9.784221,-1.009078
761832,13.0,11.398688,-1.834070


In [18]:
# Drop TaxiIns
df = df.loc[(df.TaxiIn > df.thresholdDownTaxiIn) | (df.TaxiIn < df.thresholdUpTaxiIn),:]

### Fuel

- There are `Burnoff` <= 0 with positive `FlightTime`. 12 rows that are clearly wrong imputations. Remove them.

In [19]:
df.loc[df.Burnoff <= 0,['Burnoff','FlightTime', 'BlockTime']]

,Burnoff,FlightTime,BlockTime
73792,-3970.0,107.0,125.0
167273,-2056.0,50.0,67.0
176025,-3096.0,23.0,37.0
189322,0.0,46.0,61.0
391446,-2442.0,41.0,57.0
439514,-795.0,45.0,60.0
495513,-828.0,36.0,52.0
558209,-2770.0,96.0,115.0
572439,-1577.0,45.0,54.0
726464,-569.0,76.0,93.0


In [20]:
# Drop
df = df.loc[df.Burnoff > 0,:]

### People and Bags

- For those flights that have ( `Adults` + `Children` ) > `AircraftCapacity` we "fix" the number of adults and children down to the capacity.
- First we substract children, then adults

In [21]:
# Reduce Children then Adults until match AircraftCapacity
def balancePassangers(capacity, adults, children):
    total = adults + children
    if total > capacity:
        if total - children <= capacity:
            children = children - (total-capacity)
            return adults, children
        elif total - children > 180:
            total = total - children
            children = 0
            adults = adults - (total-capacity) 
            return adults, children
    else: return adults, children

df[['Adults','Children']] = df.apply(lambda x: balancePassangers(x.AircraftCapacity, x.Adults, x.Children), axis=1, result_type='expand')

- Flights with `Adults` == 0
- All "Positioning Flights" (46) should have Adults, Children, Infants == 0. We fix 8 of them.

In [22]:
df.loc[df.ServiceDescription == 'Positioning Flight', ['Adults']] = 0
df.loc[df.ServiceDescription == 'Positioning Flight', ['Children']] = 0
df.loc[df.ServiceDescription == 'Positioning Flight', ['Infants']] = 0

- There are 903 "Scheduled Flights" with 0 Adults. 
- Since we won't be able to use weight variables in the test set, we do the following:
    - Those flights with Bags < 10 -> Classified as Positioning Flight. They have low TeledyneRampWeight, the ServiceDescription was probably wrongly inputed.
    - The rest of flights will be inputed with the median passangers for that route. Using a ratio Bags/Passangers to inpute values might be dangerours given that the Bags might also be wrong...

In [23]:
df.loc[(df.Adults == 0) & (df.Bags < 10) & (df.ServiceDescription == 'Scheduled Flight'), 'ServiceDescription'] = 'Positioning Flight'
meanAdultsRoute = df.groupby('ScheduledRoute')['Adults'].median()
df.loc[(df.Adults == 0) & (df.ServiceDescription == 'Scheduled Flight'), ['Adults']] = df['ScheduledRoute'].map(meanAdultsRoute)

- The same strategy is applied to "CharterFlights"
- "Charter Flight (Tour Operator)" is classified as "Charter Flight" - we didn't find any major difference between the two.

In [24]:
df.loc[df.ServiceDescription == 'Charter Flight (Tour Operator)', 'ServiceDescription'] = 'Charter Flight'
df.loc[(df.Adults == 0) & (df.Bags < 10) & (df.ServiceDescription == 'Charter Flight'), 'ServiceDescription'] = 'Positioning Flight'
df.loc[(df.Adults == 0) & (df.ServiceDescription == 'Charter Flight'), ['Adults']] = df['ScheduledRoute'].map(meanAdultsRoute)

- Bags outlier at 1170

In [25]:
df  = df.loc[df.Bags < 1000,]

### CARGO - FREIGHT
- Huge Outliers for Freights.
- After exploring the data, we considered everything above 10000 as an outlier.
- All flights with `Freights` > 1000 (16) are dropped

In [26]:
df = df.loc[df.Freight <= 10000,:]

### RAMP WEIGHT

- From regulation for each plane, we drop all rows than have `TeledyneRampWeight` higher than allowed
- Also, we set Min TeledyneRampWeight -> 40,000, smaller is outlier

In [27]:
# Max
df.drop(df[(df['AircraftCapacity'] == 148) & (df['TeledyneRampWeight'] > 70080)].index, inplace=True)
df.drop(df[(df['AircraftCapacity'] == 189) & (df['TeledyneRampWeight'] > 79016)].index, inplace=True)
df.drop(df[(df['AircraftCapacity'] == 180) & (df['TeledyneRampWeight'] > 73700)].index, inplace=True)
df.drop(df[(df['AircraftCapacity'] == 197) & (df['TeledyneRampWeight'] > 82191)].index, inplace=True)
# Min
df = df.loc[df.TeledyneRampWeight > 40000,:]

- Teledynes with same value all? Value 74283 is repeated 4088 and does not match PlannedTow many times.
- Looking at our initial models we saw that our predictions deviated a lot from these Teledynes, but the PlannedTOW was similiar to out estimations.
- To filter these kind of wrong values (and assuming PlannedTOW is accurate) we use `WrongTeledyneRatio`
    - Filter when PlannedTOW is more than 10% higher than Teledyne (3424 rows) 
    - Filter when PlannedTOW is less than 85% of Teledyne (6565 rows)

In [28]:
df['WrongTeledyneRatio'] = df['TeledyneRampWeight']/df['PlannedTOW']
df = df.loc[df.WrongTeledyneRatio <= 1.10,:]
df = df.loc[df.WrongTeledyneRatio >= 0.85,:]

## Feature Engineering

### Month + "Weektime"
- We extract "Month" feature
- We divide weekdays into:
    - "weekend" (friday, saturday and sunday)
    - "weekday" (monday to thursday)

In [29]:
df.DepartureScheduled = df.DepartureScheduled.astype('datetime64[ns]')
df['month'] = df['DepartureScheduled'].dt.strftime("%b")
# Weektime
df['weektime'] = df.DepartureScheduled.dt.strftime('%w')
df['weektime'] = df['weektime'].apply(lambda x: 'weekend' if x in ['5','6','0'] else 'weekday')

### "Weight Common Sense"
- We wanted to have our own common sense `PlannedZeroFuelWeight`, which we found to be relevant to our models and that is useful for other ratios
- We call it `weightCargo` and it's close to Ryanair's `PlannedZeroFuelWeight` as you can see in the describe table

In [30]:
df['weightCargo'] = df['DryEmptyWeight'] + df['Adults']*84 + df['Children']*40 + df['Infants']*20 + df['Bags']*20 + df['Freight']
(df['weightCargo']/df['PlannedZeroFuelWeight']).describe()

count    750876.000000
mean          0.947469
std           0.032184
min           0.709331
25%           0.936460
50%           0.955172
75%           0.968037
max           1.301444
dtype: float64

### Fuel Consumption
- Feature `consumption` allows us to capture `Burnoff` per minute per kg
- It's important to note that this consumption will be different for each aircraft type 

In [31]:
df['consumption'] = df['Burnoff'] / df['BlockTime'] / df['TeledyneRampWeight'] 
df.groupby(['AircraftCapacity'])['consumption'].agg(['mean','median','std'])

,mean,median,std
AircraftCapacity,,,
148.0,0.000586,0.000587,0.000050
180.0,0.000599,0.000598,0.000036
189.0,0.000575,0.000575,0.000033
197.0,0.000467,0.000466,0.000023


- Another metric than we found was very relevant in our models it `Burnoff` per Kg of weight without counting fuel, which varies per `ScheduledRoute` and `AircraftTypeGroup`
- This metric also allows to capture that if the plane has a higher "Zero Fuel Weight" we might consume more.
- Feature `burnoffKg` and we use our "common sense" `weightCargo` to compute it

In [32]:
df['burnoffKg'] = df['Burnoff'] / df['weightCargo']

### Extra Fuel
- We wanted to capture wether aircrafts carry much more fuel than the Burnoff - `extraFuelRatio`
- There are routes that on average carry more fuel in reserve, probably because they won't refuel at the arrival airport
- `fuel` carried is calculated using both features from Ryanair's (assuming that if there is an error in one the other will also be wrong)


In [33]:
df['fuel'] = df['PlannedTOW'] - df['PlannedZeroFuelWeight']
df['extraFuelRatio'] = df['Burnoff'] / df['fuel']

## Aggregating New Features for the Model
- Given that many of the new features (ratios) we created were using the *variables known after prediction* (which we don't have in our test set) we aggregated these in order to be used in our model:

    - `meanTaxiOut` aggregated per `DepartureAirport`
    - `meanConsumption` aggregated per `AircraftRegistration`
    - `meanFlightTime` aggregated per `ScheduledRoute` and `AicraftCapacity`
    - `meanBurnoff` aggregated per `ScheduledRoute` and `AicraftCapacity`
    - `meanBurnoffKg` aggregated per `ScheduledRoute` and `AicraftCapacity`
    - `meanExtraFuelRatio` aggregated per `ScheduledRoute`
    - `meanExtraFuelRatioArrival` aggregated per `ArrivalAirport`

In [34]:
meanTaxiOut = df.groupby(['DepartureAirport'])['TaxiOut'].mean()
df['meanTaxiOut'] = df['DepartureAirport'].map(meanTaxiOut)

meanConsumption = df.groupby('AircraftRegistration')['consumption'].mean()
df['meanConsumption'] = df['AircraftRegistration'].map(meanConsumption)

meanFlightTime = df.groupby(['ScheduledRoute','AircraftCapacity'], as_index=False)['FlightTime'].mean().rename({'FlightTime':'meanFlightTime'},axis=1)
df = df.merge(meanFlightTime, how='left')

meanBurnoff = df.groupby(['ScheduledRoute','AircraftCapacity'], as_index=False)['Burnoff'].mean().rename({'Burnoff':'meanBurnoff'},axis=1)
df = df.merge(meanBurnoff, how='left')

meanBurnoffKg = df.groupby(['ScheduledRoute','AircraftCapacity'], as_index=False)['burnoffKg'].mean().rename({'burnoffKg':'meanBurnoffKg'},axis=1)
df = df.merge(meanBurnoffKg, how='left')

meanExtraFuelRatio = df.groupby(['ScheduledRoute'], as_index=False)['extraFuelRatio'].mean().rename({'extraFuelRatio':'meanExtraFuelRatio'},axis=1)
df = df.merge(meanExtraFuelRatio, how='left')

meanExtraFuelRatioArrival = df.groupby(['ArrivalAirport'], as_index=False)['extraFuelRatio'].mean().rename({'extraFuelRatio':'meanExtraFuelRatioArrival'},axis=1)
df = df.merge(meanExtraFuelRatioArrival, how='left')


### More Data Cleaning?
We considered and tried cleaning more rows from the data frame by using some of these ratios. For instance, cleaning burnoffRates that were much higher than the average. However, the model did not improve as much and we were taking out outliers that might be helpful when predicting the "test set" data where we cannot identify those outliers. Hence, we chose to not clean any more rows from the dataset, although we are conscient that there are many other "wrong" inputations

# Model

### Feature Selection
- Select columns from `df` to use in the model
- One hot encoding for categorical variables

In [35]:
X = df.loc[:,[
    'AircraftCapacity',
    'AircraftTypeGroup',
    'ServiceDescription',
    'Carrier',
    'AOCDescription',
    'BlockTimeScheduled',
    'Adults',
    'Children',
    'Infants',
    'Freight',
    'Bags',
    'DryEmptyWeight',
    'distance',
    'month',
    'weektime',
    'meanTaxiOut',
    'meanConsumption',
    'meanFlightTime',
    'meanBurnoff',
    'meanBurnoffKg',
    'meanExtraFuelRatio',
    'meanExtraFuelRatioArrival'
]]

X = pd.get_dummies(X,columns=['AircraftTypeGroup','ServiceDescription','Carrier','AOCDescription', 'month','weektime'])

y = df.loc[:,['TeledyneRampWeight']]

In [36]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=15)

#parametros iniciales 540
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l1','l2'],
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 8,
    'verbose': -1,
    "max_depth": 10,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 100000,
}

gbm = lgb.LGBMRegressor(**hyper_params)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='mae',
        early_stopping_rounds=50,
        verbose=False)
gbm.best_score_

[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7


defaultdict(collections.OrderedDict,
            {'valid_0': OrderedDict([('l1', 662.3179104300796),
                          ('l2', 1054708.1361935106)])})

## Prepare the Test Set for Submission (Model in Production)

The next cells show how we prepared the test set for submission (inpute the new created features used in the mode), which is what would be needed in case of wanting to run this model in production

In [37]:
submission = pd.read_csv('test.csv')

# Calculate distance between airports
submission[['DepartureAirport','ArrivalAirport']] = submission['ScheduledRoute'].str.split('-', expand=True)
submission['distance'] = submission.apply(lambda x: calculateGeoDistance(x.DepartureAirport, x.ArrivalAirport), axis = 1)

# Month
submission.DepartureScheduled = submission.DepartureScheduled.astype('datetime64[ns]')
submission['month'] = submission['DepartureScheduled'].dt.strftime("%b")
# Weektime
submission['weektime'] = submission.DepartureScheduled.dt.strftime('%w')
submission['weektime'] = submission['weektime'].apply(lambda x: 'weekend' if x in ['5','6','0'] else 'weekday')

# Balance Passangers to AircraftCapacity
submission[['Adults','Children']] = submission.apply(lambda x: balancePassangers(x.AircraftCapacity, x.Adults, x.Children), axis=1, result_type='expand')

# Freights Max = 10,000 --> All values above 10,000 input as 10,000
submission.loc[submission.Freight > 10000, 'Freight'] = 10000

# DryEmptyWeight
dWeights = {197:45070, 189: 41140, 148:37648 ,180:42600}
submission['DryEmptyWeight'] = submission['AircraftCapacity'].apply(lambda x: dWeights[x])

# weightCargo common sense
submission['weightCargo'] = submission['DryEmptyWeight'] + submission['Adults']*84 + submission['Children']*40 + submission['Infants']*20 + submission['Bags']*20 + submission['Freight']

# ServiceDescription Corrections
submission.loc[submission.ServiceDescription == 'Positioning Flight', ['Adults']] = 0
submission.loc[submission.ServiceDescription == 'Positioning Flight', ['Children']] = 0
submission.loc[submission.ServiceDescription == 'Positioning Flight', ['Infants']] = 0

submission.loc[(submission.Adults == 0) & (submission.Bags < 10) & (submission.ServiceDescription == 'Scheduled Flight'), 'ServiceDescription'] = 'Positioning Flight'
meanAdultsRoute = df.groupby('ScheduledRoute')['Adults'].median()
submission.loc[(submission.Adults == 0) & (submission.ServiceDescription == 'Scheduled Flight'), ['Adults']] = submission['ScheduledRoute'].map(meanAdultsRoute)

submission.loc[submission.ServiceDescription == 'Charter Flight (Tour Operator)', 'ServiceDescription'] = 'Charter Flight'
submission.loc[(submission.Adults == 0) & (submission.Bags < 10) & (submission.ServiceDescription == 'Charter Flight'), 'ServiceDescription'] = 'Positioning Flight'
submission.loc[(submission.Adults == 0) & (submission.ServiceDescription == 'Charter Flight'), ['Adults']] = submission['ScheduledRoute'].map(meanAdultsRoute)


### Ratios from Feature Engineering
Since some `ScheduledRoutes` and some `Aiports` found in the test set are not in the training we need to decide which value to inpute:
- `meanTaxiOut`: For Null values inpute the median of all `TaxiOut`
- `meanConsumption`: For Null values inpute the median of all `consumption`
- `meanFlightTime`: For Null values use a LinearRegression where X=distance and y=FlightTime from the entire training set
- `meanBurnoff`: For Null values first input the mean in that `ScheduledRoute`, otherwise use a LinearRegression X=distance and y=Burnoff
- `meanBurnoffKg`: For Null values input the mean of by `AicraftCapacity`
- `meanExtraFuelRatio` and `meanExtraFuelRatioArrival`: For Null values input the mean of all extraFuelRatio


In [38]:
from sklearn.linear_model import LinearRegression

submission['meanTaxiOut'] = submission['DepartureAirport'].map(meanTaxiOut)
submission.meanTaxiOut = submission.meanTaxiOut.fillna(df.TaxiOut.median())

submission['meanConsumption'] = submission['AircraftRegistration'].map(meanConsumption)
submission.meanConsumption = submission.meanConsumption.fillna(df.consumption.median())

submission = submission.merge(meanFlightTime, how='left')
regFlightTime = LinearRegression().fit(df['distance'].array.reshape(-1, 1),df['FlightTime'].array.reshape(-1, 1))
submission.loc[submission['meanFlightTime'].isna(),'meanFlightTime'] = regFlightTime.predict(submission.loc[submission['meanFlightTime'].isna(),'distance'].array.reshape(-1, 1))

submission = submission.merge(meanBurnoff, how='left')
submission.loc[submission['meanBurnoff'].isna(),'meanBurnoff'] = submission['ScheduledRoute'].map(meanBurnoff.groupby('ScheduledRoute')['meanBurnoff'].mean())
regBurnoff = LinearRegression().fit(df['distance'].array.reshape(-1, 1),df['Burnoff'].array.reshape(-1, 1))
submission.loc[submission['meanBurnoff'].isna(),'meanBurnoff'] = regBurnoff.predict(submission.loc[submission['meanBurnoff'].isna(),'distance'].array.reshape(-1, 1))

submission = submission.merge(meanBurnoffKg, how='left')
submission.loc[submission['meanBurnoffKg'].isna(),'meanBurnoffKg'] = submission['AircraftCapacity'].map(meanBurnoffKg.groupby('AircraftCapacity')['meanBurnoffKg'].mean())

submission = submission.merge(meanExtraFuelRatio, how='left')
submission.loc[submission['meanExtraFuelRatio'].isna(),'meanExtraFuelRatio'] = meanExtraFuelRatio.meanExtraFuelRatio.mean()

submission = submission.merge(meanExtraFuelRatioArrival, how='left')
submission.loc[submission['meanExtraFuelRatioArrival'].isna(),'meanExtraFuelRatioArrival'] = meanExtraFuelRatioArrival.meanExtraFuelRatioArrival.mean()


In [39]:
Xsub = submission.loc[:,[
    'AircraftCapacity',
    'AircraftTypeGroup',
    'ServiceDescription',
    'Carrier',
    'AOCDescription',
    'BlockTimeScheduled',
    'Adults',
    'Children',
    'Infants',
    'Freight',
    'Bags',
    'DryEmptyWeight',
    'distance',
    'month',
    'weektime',
    'meanTaxiOut',
    'meanConsumption',
    'meanFlightTime',
    'meanBurnoff',
    'meanBurnoffKg',
    'meanExtraFuelRatio',
    'meanExtraFuelRatioArrival'
]]
Xsub = pd.get_dummies(Xsub,columns=['AircraftTypeGroup','ServiceDescription','Carrier','AOCDescription', 'month','weektime'], drop_first=False)

predsub = gbm.predict(Xsub)

In [40]:
subFinal = pd.concat([submission['FlightID'], pd.Series(predsub,name='TeledyneRampWeight')], axis=1)
subFinal.to_csv('subFinal.csv', index=False)